# 模型构造

## 层和块

## 回顾一下多层感知机

In [2]:
#回顾一下多层感知机
import torch
from torch import nn
from torch.nn import functional as F

#nn.Sequential定义了一个特殊的Module
net=nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

#随机生成一个2*20的矩阵，满足均匀分布
X=torch.rand(2,20)
net(X)

tensor([[-2.5866e-01,  8.3700e-02, -8.0222e-02,  2.0417e-01, -5.9994e-02,
         -7.6787e-02, -1.4708e-01, -1.8244e-01, -1.4637e-01,  2.3431e-01],
        [-2.3706e-01, -6.9505e-03, -1.0128e-01,  9.1590e-02, -9.6492e-05,
         -1.1386e-01, -1.5017e-01, -1.5856e-01, -2.0885e-01,  2.3384e-01]],
       grad_fn=<AddmmBackward>)

## 自定义MLP

In [3]:
#任何一个层和神经网络都是Module的子类
#自定义一个MLP
#MLP就可以继承Module中很多的函数

class MLP(nn.Module):
    #这个函数就用来定义一些参数和类
    #网络中调用的所有层都在这个函数里面
    def __init__(self):
        #调用父类，把一些内部参数全部设置好，这样在初始化weight可以全部弄好
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    #前向计算
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

#实例化多层感知机的层，每次调用正向传播函数时调用这些层
net=MLP()
net(X)

tensor([[ 0.0325,  0.0237, -0.0557, -0.1316,  0.1018, -0.2487, -0.0263,  0.0706,
         -0.1457, -0.1961],
        [ 0.0813, -0.0072,  0.0048, -0.0931,  0.1010, -0.3355,  0.0130,  0.0275,
         -0.0447, -0.2847]], grad_fn=<AddmmBackward>)

## 实现nn.Sequential

#实现nn.Sequential
class MySequential(nn.Module):
    def __init__(self,*args):#args里面存放各个层
        super().__init__()
        for block in args:
            #做成一个字典
            self._modules[block]=block
            
    def forward(self,X):
        #把各个层取出来，输入X得到一个输出，不断按顺序经过每一层
        for block in self._modules.values():
            X=block(X)
            #print(self._modules)
            #print(self._modules.values())
        return X
    
net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

## 在正向传播函数中执行代码

In [4]:
#在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(20,20)
        #因为设置的是False，所以是不参与求导的，不参与训练的
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        X=self.linear(X)
        while X.abs().sum() > 1:
            X/=2
        
        return X.sum()
    
net=FixedHiddenMLP()
net(X)

tensor(0.0306, grad_fn=<SumBackward0>)

## 混合搭配各种组合块的方法

In [5]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
        
    def forward(self,X):
        X=self.net(X)
        X=self.linear(X)
        
        return X   
net=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
net(X)      

tensor(-0.2247, grad_fn=<SumBackward0>)

# 参数管理

## 首先关注具有单隐藏层的多层感知机

In [6]:
import torch
from torch import nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(2,4)
net(X)

tensor([[0.2999],
        [0.3167]], grad_fn=<AddmmBackward>)

## 参数访问

In [7]:
#把每一层里面的那个权重拿出来
#nn.Sequential可以简单的理解为python的一个list
#这是拿到的最后一层的参数即权重和偏置nn.Linear(8,1)
#全连接层的参数就是权重和偏置
print(net[2].state_dict())
print(net.state_dict())

OrderedDict([('weight', tensor([[ 0.2862,  0.3521,  0.0053,  0.2196, -0.0531,  0.2587, -0.2426, -0.1172]])), ('bias', tensor([0.0831]))])
OrderedDict([('0.weight', tensor([[-0.3569, -0.4068,  0.3619,  0.3387],
        [ 0.2706, -0.1916,  0.1260,  0.2155],
        [ 0.1978, -0.4180,  0.4577, -0.3133],
        [-0.3524, -0.0809,  0.4494,  0.2720],
        [-0.4618, -0.4195,  0.2854,  0.1309],
        [-0.0341,  0.2848, -0.3511,  0.0365],
        [ 0.0949, -0.3319, -0.1185,  0.3313],
        [-0.2899, -0.2630,  0.2026, -0.0393]])), ('0.bias', tensor([ 0.1191,  0.4609,  0.3057, -0.0077,  0.1718, -0.0971,  0.1261, -0.0232])), ('2.weight', tensor([[ 0.2862,  0.3521,  0.0053,  0.2196, -0.0531,  0.2587, -0.2426, -0.1172]])), ('2.bias', tensor([0.0831]))])


In [8]:
#可以访问一些具体的参数
#访问偏置的内容
print(net[2].bias)
#访问偏置的值
print(net[2].bias.data)
#访问权重的梯度
print(net[2].bias.grad)
#查看bias的数据类型
print(type(net[2].bias))

Parameter containing:
tensor([0.0831], requires_grad=True)
tensor([0.0831])
None
<class 'torch.nn.parameter.Parameter'>


In [9]:
#访问权重的梯度
#因为还么有反向训练，所以是None
print(net[2].weight.grad)

None


### 一次性访问所有参数

In [10]:
#一次性访问所有参数
#relu没有参数，所以拿不出来
#named_parameters()是访问所有的参数
for name,param in net.named_parameters():
    print(name)
    print(param)
    print(param.shape)

#访问第0层的所有参数
for name,param in net[0].named_parameters():
    print(name)
    print(param)
    print(param.shape)

0.weight
Parameter containing:
tensor([[-0.3569, -0.4068,  0.3619,  0.3387],
        [ 0.2706, -0.1916,  0.1260,  0.2155],
        [ 0.1978, -0.4180,  0.4577, -0.3133],
        [-0.3524, -0.0809,  0.4494,  0.2720],
        [-0.4618, -0.4195,  0.2854,  0.1309],
        [-0.0341,  0.2848, -0.3511,  0.0365],
        [ 0.0949, -0.3319, -0.1185,  0.3313],
        [-0.2899, -0.2630,  0.2026, -0.0393]], requires_grad=True)
torch.Size([8, 4])
0.bias
Parameter containing:
tensor([ 0.1191,  0.4609,  0.3057, -0.0077,  0.1718, -0.0971,  0.1261, -0.0232],
       requires_grad=True)
torch.Size([8])
2.weight
Parameter containing:
tensor([[ 0.2862,  0.3521,  0.0053,  0.2196, -0.0531,  0.2587, -0.2426, -0.1172]],
       requires_grad=True)
torch.Size([1, 8])
2.bias
Parameter containing:
tensor([0.0831], requires_grad=True)
torch.Size([1])
weight
Parameter containing:
tensor([[-0.3569, -0.4068,  0.3619,  0.3387],
        [ 0.2706, -0.1916,  0.1260,  0.2155],
        [ 0.1978, -0.4180,  0.4577, -0.3133],

### 根据名字访问指定的参数

In [11]:
#由上一步输出所有参数的名字，我们可以根据名字来获取指定的参数
net.state_dict()['2.bias'].data

tensor([0.0831])

### 从嵌套块收集参数

In [24]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        #这里add_module的好处是说，可以写字符进去f'block {i}'，可以自己命名块的名字
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet(X))
print(X.shape)

tensor([[-0.0598],
        [-0.0598]], grad_fn=<AddmmBackward>)
torch.Size([2, 4])


## 通过print（net)大致了解这个网络内部什么样子

In [13]:
#可以通过print（net)大致了解这个网络内部什么样子
#三个Sequential
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 内置初始化

In [23]:
#如何去修改默认的初始函数，这里主要是修改了初始的权重和偏置
#m is Module
#nn.init里面写了很多初始化的函数
def init_normal(m):
    if type(m)==nn.Linear:
        #下划线是指不返回一个值，而是用这个值直接去改写权重，满足一个均值为0，方差为0.01的正态分布
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)

#这个apply相当于一个循环，把init_normal初始化函数应用到net里面每一个layer
net.apply(init_normal)
print(net[0].weight.data[0],net[0].bias.data[0])

#如果data不加0的话，那第0层有很多权重和偏置，就都会输出来，加0只是输出第一个
print(net[0].weight.data,net[0].bias.data)

def init_constant(m):
    if type(m)==nn.Linear:
        #下划线是指不返回一个值，而是用这个值直接去改写权重，满足一个均值为0，方差为0.01的正态分布
        #constant是指一个固定值
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)
print(net[0].weight.data[0],net[0].bias.data[0]) 
print(net[0].weight.shape,net[0].bias.shape) 

tensor([ 0.0130, -0.0017, -0.0074,  0.0083]) tensor(0.)
tensor([[ 0.0130, -0.0017, -0.0074,  0.0083],
        [ 0.0058,  0.0021, -0.0001,  0.0039],
        [-0.0080,  0.0180, -0.0065, -0.0146],
        [-0.0022, -0.0098, -0.0011, -0.0021],
        [-0.0010, -0.0012,  0.0143,  0.0098],
        [ 0.0073,  0.0086,  0.0115, -0.0022],
        [-0.0051,  0.0058,  0.0104,  0.0048],
        [ 0.0100,  0.0063,  0.0217, -0.0034]]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 0.0027,  0.0119, -0.0098,  0.0123]) tensor(0.)
torch.Size([8, 4]) torch.Size([8])


### 对某些块应用不同的初始化方法

In [22]:
#用的是vavier的uniform distribution(均匀分布)初始化
def xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)

#固定值42
def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)
        
#net可以调用apply,那么net的每一层也都可以单独调用        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.0754,  0.3403, -0.6856, -0.0308])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


## 自定义初始化